# Compute S

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import os
from glob import glob

import matplotlib.pyplot as plt

import histlib.matchup as match
import histlib.diagnosis as diag
from histlib.cstes import labels, zarr_dir, matchup_dir, var
from histlib.matchup import _data_var, _stress_var, _aviso_var

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=20, processes=20, walltime='04:00:00')
    w = cluster.scale(jobs=3)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client


KeyboardInterrupt



## Choose DL

In [3]:
DL =100*1e3 #meters
DT = 3*3600 #seconds

# Build and store dataset

In [4]:
def ms_dataset(dsm, l) :
    dsm = match.add_except_sum(dsm)   
    nb = dsm.sizes["obs"]
    ds = (dsm**2).mean('obs')
    ds["nb_coloc"] = nb
    ds['drifter_sat_year']=l
    ds = ds.expand_dims('drifter_sat_year')
    ds = ds.set_coords('drifter_sat_year')
    
    if "id_comb" in dsm:
        ds["id_comb"] = dsm["id_comb"]
    ## ATTRS
    for v in dsm.keys() :
        ds[v].attrs =dsm[v].attrs
        ds[v].attrs['units']=r'$m^2.s^{-4}$'
        
    return ds

In [ ]:
for l in labels : 
    dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr'))[var+['drogue_status', 'alti___distance', 'alti___time_difference']].dropna('obs').chunk({'obs':500})
    dsm = dsm.where(dsm.alti___distance<=DL, drop=True)
    dsm = dsm.where(dsm.alti___time_difference<=DT, drop=True).drop(['alti___distance', 'alti___time_difference'])
    dsmd = dsm.where(dsm.drogue_status, drop=True).drop('drogue_status')
    dsmnd = dsm.where(np.logical_not(dsm.drogue_status), drop=True).drop('drogue_status')
    dsm = dsm.drop('drogue_status')
    if l==labels[0] : 
        ms_dataset(dsm, l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{DL//1000}_{DT}.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
        if dsmd.dims['obs']!=0 : 
            ms_dataset(dsmd,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{DL//1000}_{DT}_drogued.zarr'), encoding={'drifter_sat_year':{'dtype':'U32'}},mode='w')
        if dsmnd.dims['obs']!=0 : 
            ms_dataset(dsmnd,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{DL//1000}_{DT}_undrogued.zarr'),encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
    else : 
        ms_dataset(dsm,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{DL//1000}_{DT}.zarr'), append_dim='drifter_sat_year')
        if dsmd.dims['obs']!=0 : 
            ms_dataset(dsmd,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{DL//1000}_{DT}_drogued.zarr'), append_dim='drifter_sat_year')
        if dsmnd.dims['obs']!=0 : 
            ms_dataset(dsmnd,l).to_zarr(os.path.join(zarr_dir+'_ok',f'ms_{DL//1000}_{DT}_undrogued.zarr'),append_dim='drifter_sat_year')
    print(l)


gps_Jason-3_2020


In [3]:
cluster.close()

tornado.application - ERROR - Exception in callback functools.partial(<function sync.<locals>.cancel at 0x2aaaf7a04700>)
Traceback (most recent call last):
  File "/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/tornado/ioloop.py", line 738, in _run_callback
    ret = callback()
  File "/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/distributed/utils.py", line 415, in cancel
    if future is not None:
NameError: free variable 'future' referenced before assignment in enclosing scope
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x2aaaf6ad35b0>>, <Task finished name='Task-18' coro=<SpecCluster._correct_state_internal() done, defined at /home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/distributed/deploy/spec.py:346> exception=RuntimeError('Command exited with non-zero exit code.\nEx

KeyboardInterrupt: 